In [61]:
import pandas as pd
import numpy as np
import heapq

# Load data
interactions = pd.read_csv('./data/interactions.csv', sep='\t')
items = pd.read_csv('./data/item_profile.csv', sep='\t')
users = pd.read_csv('./data/user_profile.csv', sep='\t')
items

,id,title,career_level,discipline_id,industry_id,country,region,latitude,longitude,employment,tags,created_at,active_during_test
0,2034642,"3948498,4306355,2410739",1.0,15,15,de,10,50.8,6.1,1,"592205,2660030",0,0
1,595759,"817205,1697930,3689751,807663,297403,99231,202...",3.0,20,1,at,0,48.4,13.4,1,"2021989,2550387,903913,1151866,1604815,1272071...",1439848800,0
2,1953291,2517971,3.0,18,15,de,3,52.5,13.4,1,"486025,4499290,2707971,692505,903913",1438293600,0
3,835222,"1057841,4528517",3.0,12,15,de,4,52.4,14.3,0,"2167118,1454092",0,0
4,76142,"2943327,1072229",3.0,15,15,de,9,51.5,6.6,1,"1072229,2943327",0,0
5,2513312,"2865345,2573697",3.0,16,11,de,7,49.9,8.7,1,"976257,4321623,494116,2131294",1436911200,0
6,270773,2117739,3.0,3,15,de,2,48.1,11.6,1,"798840,1514943,4321623,2117739",0,1
7,917719,1072848,1.0,15,15,de,2,49.0,12.1,1,"143193,2660030,1309022,3922780,2020832,3665606...",0,0
8,2129427,1491612,3.0,20,15,de,0,0.0,0.0,1,"1638389,3665606,4499767,4321623,798840,213841,...",1437775200,1
9,415377,"3083772,3408137",3.0,3,15,at,0,48.0,14.4,1,"1126746,272304,759007,798840,4321623,2426818,1...",0,0


In [79]:
# TODO: Precalculate all user-item scores.
def get_interactions_score(user_id, item_id):
    user_items = interactions[(interactions['user_id'] == user_id) & (interactions['item_id'] == item_id)]
    score = 0
    for i, row in user_items.iterrows():
        score += row['interaction_type']
    return score

def get_role_title_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    
    job_roles = user['jobroles'].split(',')
    job_titles = item['title'].split(',')
    score = 0
    for role in job_roles:
        for title in job_titles:
            if title == role:
                score += 1
                
    return score

def get_career_level_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['career_level'] == item['career_level']


def get_discipline_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['discipline_id'] == item['discipline_id']

def get_industry_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['industry_id'] == item['industry_id']

def get_regions_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['region'] == item['region']

W1 = 100
W2 = 10
W3 = 12
W4 = 10
W5 = 5
W6 = 2

def heuristic_score_for_item(user_id, item_id):
    return np.sum(np.asarray([W1, W2, W3, W4, W5, W6]) * np.asarray([
        get_interactions_score(user_id, item_id),
        get_role_title_match(user_id, item_id),
        get_career_level_match(user_id, item_id),
        get_discipline_match(user_id, item_id),
        get_industry_match(user_id, item_id),
        get_regions_match(user_id, item_id)
    ]))

# Works very slow, should precalculate scores.
def heuristic_score(user_id, rec_count=5):
    recommendations = []
    for _, row in items.iterrows():
        score = heuristic_score_for_item(user_id, row['id'])
        heapq.heappush(recommendations, score)
        
    top_scores = []
    for i in range(rec_count):
        top_scores.append(heapq.heappop(recommendations))
    return top_scores


user = users[users['user_id'] == 1731019].iloc[0]
item = items[items['id'] == 2397541].iloc[0]
# user_items = interactions[(interactions['interaction_type'] == 3)]

# print(user_items)
print(item)
print(user)


print(heuristic_score_for_item(1731019, 2397541))

# print(heuristic_score(1286))

    

id                                                              2397541
title                                                   3913136,3600555
career_level                                                          3
discipline_id                                                         5
industry_id                                                          19
country                                                              at
region                                                                0
latitude                                                             48
longitude                                                          16.2
employment                                                            0
tags                  1565617,1443376,2641471,1604815,2215972,443904...
created_at                                                   1440021600
active_during_test                                                    1
Name: 81607, dtype: object
user_id                              